In [6]:
import os
import pickle
import mlflow

import pandas as pd
import numpy as np

from sklearn.feature_extraction import DictVectorizer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import make_pipeline

In [7]:
RUN_ID = os.getenv('RUN_ID', '310fdfc815584b0db42477f1f9fa6a3b')
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

MLFLOW_TRACKING_URI = "http://127.0.1:5000"
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

logged_model = f'runs:/{RUN_ID}/model'
model = mlflow.pyfunc.load_model(logged_model)


2025/05/21 09:46:14 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - cloudpickle (current: 2.0.0, required: cloudpickle==3.1.1)
 - numpy (current: 1.21.5, required: numpy==2.0.2)
 - pandas (current: 1.4.2, required: pandas==2.2.3)
 - psutil (current: 5.8.0, required: psutil==7.0.0)
 - scikit-learn (current: 1.0.2, required: scikit-learn==1.6.1)
 - scipy (current: 1.7.3, required: scipy==1.13.1)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


ModuleNotFoundError: No module named 'numpy._core'

In [3]:


mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("green-taxi-duration")

<Experiment: artifact_location='/workspaces/mlops-zoom-camp/03-training/experiment_tracking/mlruns/7', creation_time=1747697288954, experiment_id='7', last_update_time=1747697288954, lifecycle_stage='active', name='green-taxi-duration', tags={}>

In [4]:
def read_dataframe(filename: str):
    df = pd.read_parquet(filename)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.dt.total_seconds() / 60
    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df


def prepare_dictionaries(df: pd.DataFrame):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    dicts = df[categorical + numerical].to_dict(orient='records')
    return dicts

In [5]:
df_train = read_dataframe('data/green_tripdata_2021-01.parquet')
df_val = read_dataframe('data/green_tripdata_2021-02.parquet')

target = 'duration'
y_train = df_train[target].values
y_val = df_val[target].values

dict_train = prepare_dictionaries(df_train)
dict_val = prepare_dictionaries(df_val)

In [ ]:
import numpy as np
with mlflow.start_run():
    params = dict(max_depth=20, n_estimators=100, min_samples_leaf=10, random_state=0)
    mlflow.log_params(params)

    pipeline = make_pipeline(
        DictVectorizer(),
        RandomForestRegressor(**params, n_jobs=-1)
    )

    pipeline.fit(dict_train, y_train)
    y_pred = pipeline.predict(dict_val)

    mse = mean_squared_error(y_val, y_pred)
    rmse = np.sqrt(mse)
    print(params, rmse)
    mlflow.log_metric('rmse', rmse)
    


{'max_depth': 20, 'n_estimators': 100, 'min_samples_leaf': 10, 'random_state': 0} 6.7558229919200725


2025/05/20 17:03:04 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run traveling-panda-790 at: http://127.0.0.1:5000/#/experiments/7/runs/310fdfc815584b0db42477f1f9fa6a3b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/7


In [7]:
from mlflow.tracking import MlflowClient


In [8]:
MLFLOW_TRACKING_URI = 'http://127.0.0.1:5000'
RUN_ID = '310fdfc815584b0db42477f1f9fa6a3b'

client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [ ]:
path = client.download_artifacts(run_id=RUN_ID, path='dict_vectorizer.bin')

OSError: No such file or directory: '/workspaces/mlops-zoom-camp/03-training/experiment_tracking/mlruns/7/310fdfc815584b0db42477f1f9fa6a3b/artifacts/dict_vectorizer.bin'

In [ ]:
with open(path, 'rb') as f_out:
    dv = pickle.load(f_out)

In [ ]:
dv

DictVectorizer()